##plotly widgets

In addition to creating figures by sending data and embedding the result, the plotly module offers an alternative (faster) workflow by leveraging IPython/Jupyter's communication channels.

In [1]:
# we'll import everything for the whole NB, just to be a little organized upfront
import random

import numpy as np
import plotly.plotly as py
from IPython.display import display
from IPython.html import widgets
from plotly.graph_objs import Bar, Figure, Scatter, XAxis, YAxis
from plotly.widgets import GraphWidget

:0: FutureWarning:

IPython widgets are experimental and may change in the future.



<IPython.core.display.Javascript object>

##what's a widget?

Jupyter widgets let the user interact with browser in JS. This allows the frontend JS to communicate with running Python session via callbacks. Therefore, you can use Python within a Notebook to talk to the JS code controlling the widget.

Confused? Here's a simple example of a slider.

In [2]:
int_slider = widgets.IntSlider()
display(int_slider)

##and now, some magic

We can update the slider value (this lives in JS-land) just by changing the *Python* objects value in the notebook.

When you run the following cell, the slider will update!

In [3]:
int_slider.value = random.randint(0, 100)

##and now, some *useful* magic

More importantly, we can register callbacks on the widgets so that changes in JS-land can actually *cause methods to run in Python*.

After running the following cell, moving the slider above will cause `display_value` to run and you'll (quite annoyingly) see the values of the slider displayed.

In [4]:
def display_value():
    display(int_slider.value)

int_slider.on_trait_change(display_value, 'value')

##--> slide calculator challenge <--

Create callbacks for the two sliders shown below. Changes to the first slider should update the second slider such that the second slider shows the result of sin(slider_one.value).

In [5]:
slider_one = widgets.FloatSlider(min=0, max=2*np.pi, step=.01)
slider_two = widgets.FloatSlider(min=-1, max=1, step=.01)
display(slider_one)
display(slider_two)

##a new workflow

Now, we don't need to wait for responses from Plotly, we can simply iteratively update a widget without leaving client-side operations in JS! Slick!

Let's make a blank graph and then update it.

In [6]:
# you can call GraphWidget with a preexisting graph
# however, when left alone, you just get a blank plot to build with
graph = GraphWidget()
graph

##plot

The widget `plot` call mimics the signature of the `plotly.plotly.plot` function. You may build figures as you have previously with the plotly Python module.

Let's make a bar chart!

In [7]:
data = [Bar(y=[1, 2, 3])]
graph.plot(data)

##iterative!

And remember, the hard work of communicating and setting up a plotting area is already done, so iterating on the above plot is much faster.

In [8]:
data[0]['marker'] = {'color': 'orange'}
graph.plot(data)

##--> graph widget challenge <--

Using the `challenge_graph` and the `t` and `x` arrays, create a line plot with the following qualities:

* xaxis title 'time (s)'
* yaxis title 'distance (m)'
* plot title 'newton knows best'

In [9]:
g = -9.81  # m/s^2
t = np.linspace(0, 10, 100)  # [s, ...]
x = 0.5*g*t*t  # zero initial conditions : )

challenge_graph = GraphWidget()
challenge_graph

##infinitely customizable!

Since you can create widgets to interact with the user via the frontend JS, *and* the `plotly` module gives you a way to interact with Plotly plots in JS, you can do all sorts of fun things!

One simple example is attaching a scalar to a slider widget and updating the graph on change.

In [10]:
gravity_graph = GraphWidget()
scalar_slider = widgets.FloatSlider(min=-20, max=0, step=0.1, value=-9.81)
display(scalar_slider)
display(gravity_graph)

In [11]:
t = np.linspace(0, 10, 100)  # [s, ...]
g = scalar_slider.value
x = 0.5*g*t*t  # zero initial conditions : )
figure = Figure()
figure['data'] = [Scatter(x=t, y=x)]
figure['layout'].update(title='gravity graph',
                        xaxis=XAxis(title='time (s)'),
                        yaxis=YAxis(title='distance (m)', range=[-400, 0], autorange=False))
gravity_graph.plot(figure)

def update_gravity():
    g = scalar_slider.value
    x = 0.5*g*t*t  # zero initial conditions : )
    gravity_graph.restyle({'y': [x]})

scalar_slider.on_trait_change(update_gravity, 'value')

##--> widget interaction challenge <--

Leverage widgets and the given data structure to plot the data given by selecting an option from the dropdown widget.

In [12]:
data = [Scatter(y=[random.random() for j in range(100)], name='trace_{}'.format(i)) for i in range(30)]

# hint you can use the 'options' attribute to set the dropdown options
dropdown = widgets.Dropdown()
selection_graph = GraphWidget()
display(dropdown)
display(selection_graph)

In [13]:
# here's some motivation
figure = Figure()
figure['data'].extend(data)
figure['layout'].update(showlegend=False)
selection_graph.plot(figure)

##recap

* IPython/Jupyter offers widgets so that JS things in the frontend can talk to the Python code your running
* the plotly module integrates with the widget framework and gives you methods to update plots *fast*
* you can create new widgets to add custom interactions to plots to extend usability and assist in data exploration

##challenge answers

###slide calculator challenge

You need to register a callback that updates the second slider using the `on_trait_change` widget method.

    def update_slider_two():
        slider_two.value = np.sin(slider_one.value)

    slider_one.on_trait_change(update_slider_two, 'value')
    
###graph widget challenge

You need to create a figure using the plotly module and then use the `GraphWidget.plot` method to draw it.

    figure = Figure()
    figure['data'] = [Scatter(mode='lines', x=t, y=x)]
    figure['layout'].update(title='newton knows best', xaxis=XAxis(title='time'), yaxis=YAxis(title='distance'))
    challenge_graph.plot(figure)
    
###widget interaction challenge

You need to set the dropdown options using the `dropdown.options` attribute. Then, you need to make a callback to change the `selection_graph` widget based on the current `dropdown` selection. Finally, you need to register that call back to run whenever value of the dropbox is altered.

    dropdown.options=[(e['name'], i) for i, e in enumerate(data)]

    def update_graph_selection():
        selection = dropdown.value
        visible = [i == selection for i in range(len(data))]
        selection_graph.restyle({'visible': visible})

    dropdown.on_trait_change(update_graph_selection, 'value')